# chap 01 상자로서의 변수

In [50]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [51]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [52]:
x.data = np.array(2.0)
print(x.data)

2.0


In [53]:
# ndim : dimension of np.array
x = np.array(1)
print(x.ndim)

0


In [54]:
x = np.array([1, 2, 3])
print(x.ndim)

1


In [55]:
x = np.array([[1, 2, 3], [4, 5, 6]])
print(x.ndim)

2


# chap 02 변수를 낳는 함수

In [56]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx

In [57]:
x = Variable(np.array(10))
f = Square()    # Square: Function contents included, forward redefinition
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# chap 03 함수 연결

In [58]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [59]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


# chap 04 수치 미분

In [60]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data)/(2*eps)

In [61]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [62]:
# Composite function
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


In [63]:
y.grad = np.array(1.0)  # dy/dy = 1
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256
